# 실전 문제 해결 (과소적합)

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 100

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        # 첫번째 layer는 전결합계층, batchnorm, relu형태로 구성
        self.dense1 = tf.keras.layers.Dense(32, use_bias=False)
        self.batch1 = tf.keras.layers.BatchNormalization()
        
        # 그 다음 layer부턴 DenseNet구조를 사용해서 pre-activation
        self.batch2 = tf.keras.layers.BatchNormalization()
        self.dense2 = tf.keras.layers.Dense(32, use_bias=False)
        
        self.batch3 = tf.keras.layers.BatchNormalization()
        self.dense3 = tf.keras.layers.Dense(64, use_bias=False)
        
        self.batch4 = tf.keras.layers.BatchNormalization()
        self.dense4 = tf.keras.layers.Dense(128, use_bias=False)
        

        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)

        x = self.dense1(x)
        x = self.batch1(x, training)
        x = tf.nn.relu(x)

        # pre-activation
        h = self.batch2(x, training)
        h = tf.nn.relu(h)
        h = self.dense2(h)
        x = tf.concat([x, h], axis=-1)

        h = self.batch3(x, training)
        h = tf.nn.relu(h)
        h = self.dense3(h)
        x = tf.concat([x, h], axis=-1)

        h = self.batch4(x, training)
        h = tf.nn.relu(h)
        h = self.dense4(h)
        x = tf.concat([x, h], axis=-1)
        
        return self.dense5(x)

## 데이터셋 준비


In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist   # 28x28. ==> 10 class

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

4423680/4422102 [==============================] - 0s 0us/step


## Keras API 모델 학습

In [5]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5157 - accuracy: 0.8135 - val_loss: 0.5276 - val_accuracy: 0.8011
Epoch 2/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.4008 - accuracy: 0.8549 - val_loss: 0.4260 - val_accuracy: 0.8526
Epoch 3/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3657 - accuracy: 0.8676 - val_loss: 0.4014 - val_accuracy: 0.8594
Epoch 4/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3487 - accuracy: 0.8718 - val_loss: 0.3693 - val_accuracy: 0.8672
Epoch 5/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3320 - accuracy: 0.8780 - val_loss: 0.3953 - val_accuracy: 0.8550
Epoch 6/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3200 - accuracy: 0.8819 - val_loss: 0.3468 - val_accuracy: 0.8752
Epoch 7/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3094 - accuracy: 0.8845 - val_loss: 0.3621 - val_ac